In [1]:
from tqdm import tqdm
import os
import skimage
import skimage.io
import skimage.transform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

%matplotlib inline

In [2]:
indoor = 'indoor/'
outdoor = 'outdoor/'

In [3]:
files_indoor = os.listdir(indoor)
files_outdoor = os.listdir(outdoor)

In [4]:
len(files_indoor), len(files_outdoor)

(28106, 27216)

In [5]:
f_indoor = [indoor+i for i in files_indoor]
f_outdoor = [outdoor+i for i in files_outdoor]
indoor_images = skimage.io.imread_collection(f_indoor)
outdoor_images = skimage.io.imread_collection(f_outdoor)
assert len(indoor_images) == len(files_indoor)
assert len(outdoor_images) == len(files_outdoor)

In [6]:
indoor_images[0].shape

(100, 74, 3)

In [12]:
width = [image.shape[0] for image in indoor_images]
height = [image.shape[1] for image in indoor_images]

In [18]:
width_outdoor = [image.shape[0] for image in outdoor_images]
height_outdoor = [image.shape[1] for image in outdoor_images]

## 1. `Resize` изображений

In [7]:
transofrmed_image_indoor = [skimage.transform.resize(i, (32, 32)) for i in tqdm(indoor_images)]

100%|████████████████████████████████████████████████████████████████████████████| 28106/28106 [05:26<00:00, 86.06it/s]


In [8]:
transofrmed_image_indoor_2 = np.array(transofrmed_image_indoor, dtype=np.float32)

In [11]:
transofrmed_image_outdoor = [skimage.transform.resize(i, (32, 32)) for i in tqdm(outdoor_images)]

100%|███████████████████████████████████████████████████████████████████████████| 27216/27216 [03:38<00:00, 124.63it/s]


In [12]:
transofrmed_image_outdoor_2 = np.array(transofrmed_image_outdoor, dtype=np.float32)

In [13]:
transofrmed_image_indoor_2.shape, transofrmed_image_outdoor_2.shape

((28106, 32, 32, 3), (27216, 32, 32, 3))

In [14]:
X = np.concatenate((transofrmed_image_indoor_2, transofrmed_image_outdoor_2), axis=0)

In [15]:
X.shape

(55322, 32, 32, 3)

In [16]:
y = np.r_[np.ones(len(files_indoor)), np.zeros(len(files_outdoor))]

In [17]:
X.shape, y.shape

((55322, 32, 32, 3), (55322,))

## `2`. Keras

In [18]:
num_classes = 2
y_categorical = keras.utils.to_categorical(y, num_classes)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [22]:
model_1 = Sequential()
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=X_train.shape[1:]))
model_1.add(Activation('relu'))
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 32)        2432      
_________________________________________________________________
activation (Activation)      (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 32)          25632     
_________________________________________________________________
activation_1 (Activation)    (None, 6, 6, 32)          0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)              

In [24]:
batch_size = 32
opt = keras.optimizers.RMSprop(lr=0.0005, decay=1e-6)
model_1.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model_1.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=15,
              validation_data=(X_test, y_test),
              shuffle=True)

Epoch 1/15
1211/1211 [==============================] - 97s 75ms/step - loss: 0.4260 - accuracy: 0.7972 - val_loss: 0.2623 - val_accuracy: 0.8937
Epoch 2/15
1211/1211 [==============================] - 95s 79ms/step - loss: 0.2646 - accuracy: 0.8900 - val_loss: 0.2218 - val_accuracy: 0.9144
Epoch 3/15
1211/1211 [==============================] - 95s 79ms/step - loss: 0.2393 - accuracy: 0.9070 - val_loss: 0.2381 - val_accuracy: 0.9087
Epoch 4/15
1211/1211 [==============================] - 94s 78ms/step - loss: 0.2218 - accuracy: 0.9132 - val_loss: 0.2211 - val_accuracy: 0.9168
Epoch 5/15
1211/1211 [==============================] - 94s 78ms/step - loss: 0.2128 - accuracy: 0.9180 - val_loss: 0.2651 - val_accuracy: 0.8915
Epoch 6/15
1211/1211 [==============================] - 92s 76ms/step - loss: 0.2018 - accuracy: 0.9239 - val_loss: 0.2108 - val_accuracy: 0.9259
Epoch 7/15
1211/1211 [==============================] - 91s 75ms/step - loss: 0.1964 - accuracy: 0.9247 - val_loss: 0.2473 -

Получили прирост в качестве: `accuracy` $\approx 93\%$